# Model Selection 
Simplied Version - last version 0425

In [1]:
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

import sys
sys.path.append('./cs16')
import cs16.prep as prep16
import cs16.plot as plot16
import cs16.build as build16
imagesize = 64

import pandas as pd

import cs16.prep as prep16
import cs16.plot as plot16
import cv2
import numpy as np
import pandas as pd
import os
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.regularizers import l2
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
max_words=10000
max_len=100
imagesize =64


#data_type = input("Enter '1. single' for single dataset or '2. multi' for multiple dataset: ")
data_type = '1'
if data_type == '1':
    file_path = 'single.txt'
    folder_path = './data/MVSA/single/'
elif data_type == '2':
    file_path = 'multi.txt'
    folder_path = './data/MVSA/multiple/'
else:
    print("Invalid input. Please enter either 'single' or 'multi'.")
    exit()

df = pd.read_csv(file_path, index_col=None, encoding='ISO-8859-1')

Enter '1. single' for single dataset or '2. multi' for multiple dataset: 1


In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import matplotlib.pyplot as plt

# Download stopwords and punkt tokenizer if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Define a function to preprocess text
def nlp_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove URLs using regex
    text = re.sub(r'http\S+', '', text)
    
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize text into individual words
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Stem words using Porter Stemmer
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    
    # Join words back into a single string
    text = ' '.join(words)
    
    return text

# Apply the preprocess_text function to the 'tweet' column of the dataframe
df['tweet'] = df['tweet'].apply(nlp_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ausco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ausco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import xgboost as xgb
import time
# stopwatch
def some_function():
    result = 0
    for i in range(1000000):
        result += i
    return result

In [4]:

X_text, y_text = prep16.preprocess_text(df)
X_train_text, X_val_text, X_test_text, \
y_train_text, y_val_text, y_test_text = prep16.split_data(X_text, y_text, random_state=42)

X_polar, y_polar = prep16.preprocess_text(df,label = 'polarity')
X_train_polar, X_val_polar, X_test_polar, \
y_train_polar, y_val_polar, y_test_polar = prep16.split_data(X_polar, y_polar, random_state=42)

image_data_s, image_label_s = prep16.preprocess_images(df, folder_path, imagesize)
y_s = to_categorical(image_label_s, num_classes=3)

X_train_image, X_val_image, X_test_image, \
y_train_image, y_val_image, y_test_image= prep16.split_data(image_data_s, y_s, random_state=42)

y_train = to_categorical(y_train_polar, num_classes=3)
y_val =to_categorical(y_val_polar, num_classes=3)
y_test =to_categorical(y_test_polar, num_classes=3)
file_path

'single.txt'

# Model　Selection

- 單純合併 （last tested: 2025/05/09）


In [215]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import ExtraTreesClassifier
#------------------------------------------------------------
# after split
# X_train_text, X_val_text, X_test_text,   y_train_text, y_val_text, y_test_text
# X_train_image, X_val_image, X_test_image, y_train_image, y_val_image, y_test_image
# Start to count time：
#------------------------------------------------------------
import time
start_time = time.time()
#------------------------------------------------------------
########################################
# 1. one-hot 
########################################
y_train_int = np.argmax(y_train, axis=1)
y_val_int   = np.argmax(y_val,   axis=1)
y_test_int  = np.argmax(y_test,  axis=1)

########################################
# 2. Text Modality：Original RandomForest-> ExtraTree，Prob Vector(3-dim)
########################################
rf_text = LGBMClassifier(
    n_estimators=80,
    max_depth=4,
    num_leaves=8,
    learning_rate=0.1,
    boosting_type='gbdt',
    random_state=42
)
#
rf_text.fit(X_train_text, y_train_int)

# Each sample will get [p_neg, p_neu, p_pos]
train_probs_text = rf_text.predict_proba(X_train_text)  # shape -> (n_train, 3)
val_probs_text   = rf_text.predict_proba(X_val_text)    # shape -> (n_val,   3)
test_probs_text  = rf_text.predict_proba(X_test_text)   # shape -> (n_test,  3)

########################################
# 3. Image Modality：
########################################
X_train_img_flat = X_train_image.reshape(X_train_image.shape[0], -1)
X_val_img_flat   = X_val_image.reshape(X_val_image.shape[0],   -1)
X_test_img_flat  = X_test_image.reshape(X_test_image.shape[0], -1)

rf_image = LGBMClassifier(
    n_estimators=80,
    max_depth=4,
    num_leaves=8,
    learning_rate=0.1,
    boosting_type='gbdt',
    random_state=42
)

rf_image.fit(X_train_img_flat, y_train_int)

train_probs_image = rf_image.predict_proba(X_train_img_flat)  # (n_train, 3)
val_probs_image   = rf_image.predict_proba(X_val_img_flat)    # (n_val,   3)
test_probs_image  = rf_image.predict_proba(X_test_img_flat)   # (n_test,  3)

########################################
# 4. Global Fusion：Concatenate Text(3) + Image(3) -> 6 dims
########################################
X_train_global = np.concatenate([train_probs_text, train_probs_image], axis=1)  # (n_train, 6)
X_val_global   = np.concatenate([val_probs_text,   val_probs_image],   axis=1)  # (n_val,   6)
X_test_global  = np.concatenate([test_probs_text,  test_probs_image],  axis=1)  # (n_test,  6)

print("Global Fusion Feature shape (train):", X_train_global.shape)

########################################
# 5. Final Classifier
########################################
global_clf =  xgb.XGBClassifier(350, max_depth=5, random_state=42)#LogisticRegression(max_iter=500, random_state=42)
global_clf.fit(X_train_global, y_train_int)

# Validation
val_pred = global_clf.predict(X_val_global)
print("Validation Accuracy:", accuracy_score(y_val_int, val_pred))
print("Validation Classification Report:")
print(classification_report(y_val_int, val_pred,
                            digits=4,
                            target_names=['negative','neutral','positive']))

# Test
test_pred = global_clf.predict(X_test_global)
print("Test Accuracy:", accuracy_score(y_test_int, test_pred))
print("Test Classification Report:")
print(classification_report(y_test_int, test_pred,
                            digits=4,
                            target_names=['negative','neutral','positive']))

#------------------------------------------------------------

output = some_function()
print(f"Function: {output}")
end_time = time.time()
elapsed_time = end_time - start_time

# TIME TAKEN
print(f"Time Executed ===============>：{elapsed_time:.4f} Sec")

Global Fusion Feature shape (train): (3895, 6)


C:\Users\ausco\anaconda3\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


Validation Accuracy: 0.48459958932238195
Validation Classification Report:
              precision    recall  f1-score   support

    negative     0.2623    0.1778    0.2119        90
     neutral     0.5857    0.7068    0.6405       266
    positive     0.3048    0.2443    0.2712       131

    accuracy                         0.4846       487
   macro avg     0.3842    0.3763    0.3746       487
weighted avg     0.4503    0.4846    0.4620       487

Test Accuracy: 0.5297741273100616
Test Classification Report:
              precision    recall  f1-score   support

    negative     0.2182    0.1558    0.1818        77
     neutral     0.6164    0.7050    0.6577       278
    positive     0.4386    0.3788    0.4065       132

    accuracy                         0.5298       487
   macro avg     0.4244    0.4132    0.4153       487
weighted avg     0.5052    0.5298    0.5144       487

Function: 499999500000
Time Executed ===============>：53.7666 Sec


# Extra Tree Copy

In [203]:
global_clf =  xgb.XGBClassifier(350, max_depth=5, random_state=42)#LogisticRegression(max_iter=500, random_state=42)
global_clf.fit(X_train_global, y_train_int)

# Validation
val_pred = global_clf.predict(X_val_global)
print("Validation Accuracy:", accuracy_score(y_val_int, val_pred))
print("Validation Classification Report:")
print(classification_report(y_val_int, val_pred,
                            digits=4,
                            target_names=['negative','neutral','positive']))

# Test
test_pred = global_clf.predict(X_test_global)
print("Test Accuracy:", accuracy_score(y_test_int, test_pred))
print("Test Classification Report:")
print(classification_report(y_test_int, test_pred,
                            digits=4,
                            target_names=['negative','neutral','positive']))
# TIME TAKEN
print(f"Time Executed =====ExtraTree==========>：{elapsed_time:.4f} Sec")

C:\Users\ausco\anaconda3\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


Validation Accuracy: 0.5092402464065708
Validation Classification Report:
              precision    recall  f1-score   support

    negative     0.2500    0.0778    0.1186        90
     neutral     0.6000    0.7331    0.6599       266
    positive     0.3433    0.3511    0.3472       131

    accuracy                         0.5092       487
   macro avg     0.3978    0.3873    0.3752       487
weighted avg     0.4663    0.5092    0.4757       487

Test Accuracy: 0.5297741273100616
Test Classification Report:
              precision    recall  f1-score   support

    negative     0.4375    0.0909    0.1505        77
     neutral     0.6164    0.7050    0.6577       278
    positive     0.3595    0.4167    0.3860       132

    accuracy                         0.5298       487
   macro avg     0.4711    0.4042    0.3981       487
weighted avg     0.5184    0.5298    0.5039       487

Time Executed =====ExtraTree==========>：6.5392 Sec


# LGBM COPY

In [197]:
global_clf =  xgb.XGBClassifier(350, max_depth=5, random_state=42)#LogisticRegression(max_iter=500, random_state=42)
global_clf.fit(X_train_global, y_train_int)

# Validation
val_pred = global_clf.predict(X_val_global)
print("Validation Accuracy:", accuracy_score(y_val_int, val_pred))
print("Validation Classification Report:")
print(classification_report(y_val_int, val_pred,
                            digits=4,
                            target_names=['negative','neutral','positive']))

# Test
test_pred = global_clf.predict(X_test_global)
print("Test Accuracy:", accuracy_score(y_test_int, test_pred))
print("Test Classification Report:")
print(classification_report(y_test_int, test_pred,
                            digits=4,
                            target_names=['negative','neutral','positive']))
# TIME TAKEN
print(f"Time Executed =====LGBM==========>：{elapsed_time:.4f} Sec")

C:\Users\ausco\anaconda3\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


Validation Accuracy: 0.5728952772073922
Validation Classification Report:
              precision    recall  f1-score   support

    negative     0.5000    0.0556    0.1000        90
     neutral     0.5837    0.9436    0.7213       266
    positive     0.4894    0.1756    0.2584       131

    accuracy                         0.5729       487
   macro avg     0.5244    0.3916    0.3599       487
weighted avg     0.5429    0.5729    0.4820       487

Test Accuracy: 0.5811088295687885
Test Classification Report:
              precision    recall  f1-score   support

    negative     0.6000    0.0390    0.0732        77
     neutral     0.5990    0.9029    0.7202       278
    positive     0.4603    0.2197    0.2974       132

    accuracy                         0.5811       487
   macro avg     0.5531    0.3872    0.3636       487
weighted avg     0.5616    0.5811    0.5033       487

Time Executed =====LGBM==========>：189.1635 Sec


# ML Selection

In [216]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
#------------------------------------------------------------
# after split
# X_train_text, X_val_text, X_test_text,   y_train_text, y_val_text, y_test_text
# X_train_image, X_val_image, X_test_image, y_train_image, y_val_image, y_test_image
# Start to count time：
#------------------------------------------------------------
import time
start_time = time.time()
#------------------------------------------------------------
########################################
# 1. one-hot 
########################################
y_train_int = np.argmax(y_train, axis=1)
y_val_int   = np.argmax(y_val,   axis=1)
y_test_int  = np.argmax(y_test,  axis=1)

########################################
# 2. Text Modality：Original RandomForest-> ExtraTree，Prob Vector(3-dim)
########################################
rf_text = xgb.XGBClassifier(100, max_depth=5, random_state=42)
#RandomForestClassifier(n_estimators=100, random_state=42)
rf_text.fit(X_train_text, y_train_int)

# Each sample will get [p_neg, p_neu, p_pos]
train_probs_text = rf_text.predict_proba(X_train_text)  # shape -> (n_train, 3)
val_probs_text   = rf_text.predict_proba(X_val_text)    # shape -> (n_val,   3)
test_probs_text  = rf_text.predict_proba(X_test_text)   # shape -> (n_test,  3)

########################################
# 3. Image Modality：
########################################
X_train_img_flat = X_train_image.reshape(X_train_image.shape[0], -1)
X_val_img_flat   = X_val_image.reshape(X_val_image.shape[0],   -1)
X_test_img_flat  = X_test_image.reshape(X_test_image.shape[0], -1)

rf_image = xgb.XGBClassifier(100, max_depth=5, random_state=42)

rf_image.fit(X_train_img_flat, y_train_int)

train_probs_image = rf_image.predict_proba(X_train_img_flat)  # (n_train, 3)
val_probs_image   = rf_image.predict_proba(X_val_img_flat)    # (n_val,   3)
test_probs_image  = rf_image.predict_proba(X_test_img_flat)   # (n_test,  3)

########################################
# 4. Global Fusion：Concatenate Text(3) + Image(3) -> 6 dims
########################################
X_train_global = np.concatenate([train_probs_text, train_probs_image], axis=1)  # (n_train, 6)
X_val_global   = np.concatenate([val_probs_text,   val_probs_image],   axis=1)  # (n_val,   6)
X_test_global  = np.concatenate([test_probs_text,  test_probs_image],  axis=1)  # (n_test,  6)

print("Global Fusion Feature shape (train):", X_train_global.shape)

########################################
# 5. Final Classifier
########################################
global_clf =  xgb.XGBClassifier(350, max_depth=5, random_state=42)#LogisticRegression(max_iter=500, random_state=42)
global_clf.fit(X_train_global, y_train_int)

# Validation
val_pred = global_clf.predict(X_val_global)
print("Validation Accuracy:", accuracy_score(y_val_int, val_pred))
print("Validation Classification Report:")
print(classification_report(y_val_int, val_pred,
                            digits=4,
                            target_names=['negative','neutral','positive']))

# Test
test_pred = global_clf.predict(X_test_global)
print("Test Accuracy:", accuracy_score(y_test_int, test_pred))
print("Test Classification Report:")
print(classification_report(y_test_int, test_pred,
                            digits=4,
                            target_names=['negative','neutral','positive']))

#------------------------------------------------------------

output = some_function()
print(f"Function: {output}")
end_time = time.time()
elapsed_time = end_time - start_time

# TIME TAKEN
print(f"Time Executed ===============>：{elapsed_time:.4f} Sec")

C:\Users\ausco\anaconda3\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


Global Fusion Feature shape (train): (3895, 6)
Validation Accuracy: 0.5338809034907598
Validation Classification Report:
              precision    recall  f1-score   support

    negative     0.3750    0.0667    0.1132        90
     neutral     0.5732    0.8835    0.6953       266
    positive     0.3115    0.1450    0.1979       131

    accuracy                         0.5339       487
   macro avg     0.4199    0.3651    0.3355       487
weighted avg     0.4662    0.5339    0.4539       487

Test Accuracy: 0.5749486652977412
Test Classification Report:
              precision    recall  f1-score   support

    negative     0.3750    0.0390    0.0706        77
     neutral     0.5980    0.8669    0.7078       278
    positive     0.4737    0.2727    0.3462       132

    accuracy                         0.5749       487
   macro avg     0.4822    0.3929    0.3748       487
weighted avg     0.5291    0.5749    0.5090       487

Function: 499999500000
Time Executed ===============>：3

# SVM Copy

In [201]:
########################################
# 5. Final Classifier
########################################
global_clf =  xgb.XGBClassifier(350, max_depth=5, random_state=42)#LogisticRegression(max_iter=500, random_state=42)
global_clf.fit(X_train_global, y_train_int)

# Validation
val_pred = global_clf.predict(X_val_global)
print("Validation Accuracy:", accuracy_score(y_val_int, val_pred))
print("Validation Classification Report:")
print(classification_report(y_val_int, val_pred,
                            digits=4,
                            target_names=['negative','neutral','positive']))

# Test
test_pred = global_clf.predict(X_test_global)
print("Test Accuracy:", accuracy_score(y_test_int, test_pred))
print("Test Classification Report:")
print(classification_report(y_test_int, test_pred,
                            digits=4,
                            target_names=['negative','neutral','positive']))
print(f"Time Executed ==SVM=============>：{elapsed_time:.4f} Sec")

C:\Users\ausco\anaconda3\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


Validation Accuracy: 0.3264887063655031
Validation Classification Report:
              precision    recall  f1-score   support

    negative     0.1719    0.3667    0.2340        90
     neutral     0.6154    0.3008    0.4040       266
    positive     0.2788    0.3511    0.3108       131

    accuracy                         0.3265       487
   macro avg     0.3553    0.3395    0.3163       487
weighted avg     0.4429    0.3265    0.3475       487

Test Accuracy: 0.3839835728952772
Test Classification Report:
              precision    recall  f1-score   support

    negative     0.2081    0.4675    0.2880        77
     neutral     0.6042    0.3129    0.4123       278
    positive     0.3765    0.4848    0.4238       132

    accuracy                         0.3840       487
   macro avg     0.3962    0.4218    0.3747       487
weighted avg     0.4798    0.3840    0.3958       487

Time Executed ==SVM=============>：1065.0313 Sec


# Logit Copy

In [199]:
########################################
# 5. Final Classifier
########################################
global_clf =  xgb.XGBClassifier(350, max_depth=5, random_state=42)#LogisticRegression(max_iter=500, random_state=42)
global_clf.fit(X_train_global, y_train_int)

# Validation
val_pred = global_clf.predict(X_val_global)
print("Validation Accuracy:", accuracy_score(y_val_int, val_pred))
print("Validation Classification Report:")
print(classification_report(y_val_int, val_pred,
                            digits=4,
                            target_names=['negative','neutral','positive']))

# Test
test_pred = global_clf.predict(X_test_global)
print("Test Accuracy:", accuracy_score(y_test_int, test_pred))
print("Test Classification Report:")
print(classification_report(y_test_int, test_pred,
                            digits=4,
                            target_names=['negative','neutral','positive']))
print(f"Time Executed ==LOGIT=============>：{elapsed_time:.4f} Sec")

C:\Users\ausco\anaconda3\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


Validation Accuracy: 0.45790554414784396
Validation Classification Report:
              precision    recall  f1-score   support

    negative     0.2838    0.2333    0.2561        90
     neutral     0.5911    0.5977    0.5944       266
    positive     0.2986    0.3282    0.3127       131

    accuracy                         0.4579       487
   macro avg     0.3912    0.3864    0.3877       487
weighted avg     0.4556    0.4579    0.4561       487

Test Accuracy: 0.45790554414784396
Test Classification Report:
              precision    recall  f1-score   support

    negative     0.1618    0.1429    0.1517        77
     neutral     0.5788    0.6079    0.5930       278
    positive     0.3386    0.3258    0.3320       132

    accuracy                         0.4579       487
   macro avg     0.3597    0.3588    0.3589       487
weighted avg     0.4477    0.4579    0.4525       487

Time Executed ==LOGIT=============>：6.8198 Sec
